In [2]:
%matplotlib notebook
import json
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seq2seq
from seq2seq.models import Seq2Seq
from seq2seq.models import SimpleSeq2Seq
from seq2seq.models import AttentionSeq2Seq


In [7]:
model = Seq2Seq(batch_input_shape=(16, 7, 5), hidden_dim=500, output_length=8, output_dim=300, depth=4)
model.compile(loss='mse', optimizer='rmsprop')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_176 (InputLayer)          (16, 7, 5)           0                                            
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (16, 7, 500)         3000        input_176[0][0]                  
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
private__o

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_109 (InputLayer)          (16, 7, 5)           0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (16, 7, 10)          60          input_109[0][0]                  
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
private__o